# Settings

In [1]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

env: TF_KERAS=1
\


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


# Dataset loading

In [3]:
dataset_name='pokemon'

In [4]:
IMG_DIR = 'C:\\Users\\Khalid\\Documents\projects\\pokemon\DS06\\'
VAL_PCT = 10
VAL_FORMAT = 'png'

In [5]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')

In [6]:
from training.generators.file_image_generator import create_image_lists, get_generators

Using TensorFlow backend.


In [7]:
imgs_list = create_image_lists(
    image_dir=IMG_DIR, 
    validation_pct=VAL_PCT, 
    valid_imgae_formats=VAL_FORMAT
)

  DEBUG    | Looking for images in 'all'


  INFO     | 809 file found


In [8]:
inputs_shape= IMG_SIZE=(200, 200, 3)
BATCH_SIZE = 32
latent_dim = 32
intermediate_dim = 16

In [9]:
train_gen, test_gen = get_generators(
    images_list=imgs_list, 
    image_dir=IMG_DIR, 
    image_size=IMG_SIZE, 
    batch_size=BATCH_SIZE, 
    class_mode=None
)

  INFO     | Found 738 training files


  INFO     | Found 71 validation files


In [10]:
import tensorflow as tf

In [11]:
train_ds = tf.data.Dataset.from_generator(
    lambda: train_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

test_ds = tf.data.Dataset.from_generator(
    lambda: test_gen, 
    output_types=tf.float32 ,
    output_shapes=tf.TensorShape((BATCH_SIZE, ) + IMG_SIZE)
)

In [12]:
_instance_scale=1.0
for data in train_ds:
    _instance_scale = float(data[0].numpy().max())
    break

In [13]:
_instance_scale

1.0

In [14]:
import numpy as np
from collections.abc import Iterable

In [15]:
if isinstance(inputs_shape, Iterable):
    _outputs_shape = np.prod(inputs_shape)

In [16]:
_outputs_shape

120000

# Model's Layers definition

In [17]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latent_dim)]

dec_lays = [tf.keras.layers.Dense(units=latent_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [18]:
model_name = dataset_name+'ITAAEsharp_diff'
recoding_dir='recoding'+sep_local+model_name

In [19]:
from training.inputs_adversarial.autoencoders.AAE import AAE as AE

In [20]:
inputs_shape=IMG_SIZE

In [21]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latent_dim,
        'layers': enc_lays#enc_lays2#
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latent_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays#dec_lays2#
    }
]

In [22]:
from utils.data_and_files.file_utils import create_if_not_exist

In [23]:
_restore = os.path.join(recoding_dir, 'var_save_dir')

In [24]:
create_if_not_exist(_restore)
_restore

'recoding\\pokemonIAAEsharp_diff\\var_save_dir'

In [25]:
#to restore trained model, set filepath=_restore

In [ ]:
from stats.traditional_adversarial_losses import \
    create_inputs_adversarial_real_losses, \
    create_inputs_adversarial_fake_losses, \
    create_inputs_adversarial_losses

inputs_adversarial_losses = {
    'inputs_adversarial_losses': create_inputs_adversarial_losses,
    'inputs_adversarial_real_losses': create_inputs_adversarial_real_losses,
    'inputs_adversarial_fake_losses': create_inputs_adversarial_fake_losses,
}

In [1]:
ae = AE( 
    name=model_name,
    adversarial_losses=inputs_adversarial_losses,
    inputs_shape=inputs_shape,
    outputs_shape=inputs_shape,
    latent_dim=latent_dim,
    batch_size=BATCH_SIZE,
    variables_params=variables_params, 
    filepath=None
    )

NameError: name 'AE' is not defined

In [27]:
#from evaluation.quantitive_metrics.psnr import psnr
#from evaluation.quantitive_metrics.ssmi import ssmi
from evaluation.quantitive_metrics.sharp_diff import sharp_diff


In [28]:
ae.compile(loss={'x_logits': sharp_diff})

Model: "pokemonIAAEsharp_diff"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
inference (Model)            (None, 32)                20480224  
_________________________________________________________________
generative (Model)           (None, 200, 200, 3)       2041596   
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 200, 200, 3)]     0         
Total params: 22,521,820
Trainable params: 22,521,750
Non-trainable params: 70
_________________________________________________________________
None


# Callbacks

In [29]:
from training.callbacks.progress_bar import NotebookPrograssBar
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [30]:
progbar = NotebookPrograssBar(leave_outer=False)

In [31]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-12, 
    patience=5, 
    verbose=1, 
    restore_best_weights=True
)

In [32]:
ms = ModelSaver(filepath=_restore,save_freq=5)

In [33]:
csv_dir = os.path.join(recoding_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'recoding\\pokemonIAAEsharp_diff\\csv_dir\\pokemonIAAEsharp_diff.csv'

In [34]:
image_gen_dir = os.path.join(recoding_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [35]:
sg = SampleGeneration(latent_shape=latent_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [36]:
import numpy as np

# Model Training

In [37]:
ae.fit(
    x=train_ds,
    input_kw=None,
    steps_per_epoch=100,
    epochs=int(1e6), 
    verbose=0,
    callbacks=[progbar, es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=test_ds,
    validation_steps=100
)


training traditional basicAE


Model: "inputs_real_discriminator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 200, 200, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 200, 200, 16)      64        
_________________________________________________________________
flatten (Flatten)            (None, 640000)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                20480032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
activity_regularization (Act (None, 32)  

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001811CC0F988>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001812950A448>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001811CC0F988>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001812950A448>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
Instructions for updating:
If using Keras pass *_constraint 

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001811CC0F988>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001818488ED08>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001812950A448>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x000001811CC0F988>, (200, 200, 3), [50, 32], TensorSpec(shape=(50, 32), dtype=tf.float32, name='eps')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<